# Trabajo 1

Elena Torró Martínez

- [1. Iniciar Redis](#Iniciar-Redis)

## Iniciar Redis

In [1]:
import redis

host='127.0.0.1'
port=6379

redis_db=redis.Redis(host=host,port=port, decode_responses=True)

redis_db.flushall()

True

In [2]:
import pandas as pd
import numpy as np

In [3]:
relations_df = pd.read_csv('relations.csv')
relations_df.head()

,User,Follows,Following_Time
0,roxanefeller,cathcooney,13 Jun 2019 05:59:58
1,andyglittle,charleskod,14 Jul 2019 10:07:29
2,andyglittle,seers_helen,18 Jul 2019 09:50:48
3,andyglittle,karin_stowell,31 Aug 2019 15:20:48
4,hanyshita,andyglittle,12 Jul 2019 14:20:46


In [4]:
users_df = pd.read_csv('twitter_sample.csv')
users_df.head()

,User,Post_Time,Tweet_Content,Unnamed: 3
0,andyglittle,13 Jul 2019 05:59:58,We've loved being motivated by the stories of ...,NaN
1,andyglittle,05 Jul 2019 10:07:29,Thanks for the shout-out on our #MorethanMedic...,NaN
2,afparron,11 Jul 2019 09:50:48,#MorethanMedicine — Our Story via ⁦@animalhea...,NaN
3,drshahrul80,03 Aug 2019 15:20:48,We hope to get some lovely weather on our annu...,NaN
4,karin_stowell,04 Aug 2019 14:20:46,This is what #MoreThanMedicine is about. Love ...,NaN


In [5]:
import datetime
date_format = '%d %b %Y %H:%M:%S'

def date_to_datetime(date_str):
    return datetime.datetime.strptime(date_str, date_format).timestamp()

def datetime_to_date(date_datetime):
    return datetime.datetime.fromtimestamp(date_datetime).strftime(date_format)

Diseño

- `users`
- `tweets`

## Funciones

`nuevo_usuario`

Esta función recibirá el nombre del nuevo usuario y generará una nueva entrada en la base de datos utilizando un identificador incremental (proporcionado por Redis) para cada usuario. Este identificador incremental se puede utilizar para diferenciar cada una de las claves que contengan usuarios, de tal manera que dichas claves sean similares a la siguiente: “user:[id]”, donde “id” es el identificador del usuario. Para cada usuario deberemos almacenar su nombre. De la misma forma se recomienda almacenar todos los usuarios y sus identificadores dentro de una misma estructura de datos única cuya clave puede ser “users”.

In [6]:
def nuevo_usuario(username):
    if redis_db.sadd('users', username):
        new_id = redis_db.incr('user_id')
        user_id = 'user:{0}'.format(new_id)

        user_data = {
            'username': username
        }
    
        redis_db.set(username, user_id)
        redis_db.hmset(user_id, user_data)

        return user_data

    else:
        print('ERROR: User {0} already exists'.format(username))
        return False

In [7]:
def nuevo_follow(username, follow_username, date_str, follow_type):
    if username == follow_username:
        print('ERROR: Username and follower username are the same')
        return False
    if not redis_db.get(username):
        print('ERROR: User {0} does not exist'.format(username))
        return False
    if not redis_db.get(follow_username):
        print('ERROR: User {0} does not exist'.format(follow_username))
        return False

    user_id = redis_db.mget(username)[0]
    follow_id = redis_db.mget(follow_username)[0]
    user_follow_id = '{0}:{1}'.format(follow_type, user_id)

    timestamp = date_to_datetime(date_str)
    follow_data = {follow_username: timestamp}

    redis_db.zadd(user_follow_id, follow_data)
    
    return follow_data

In [8]:
def nuevo_follower(username, follower_username, date_str):
    return nuevo_follow(username, follower_username, date_str, 'followers')

In [9]:
def nuevo_following(username, follower_username, date_str):
    return nuevo_follow(username, follower_username, date_str, 'following')

In [10]:
def seguir(username, follow_username, date_str):
    nuevo_follower(username, follow_username, date_str)
    nuevo_following(follow_username, username, date_str)

In [11]:
def nuevo_post(username, message, date_str):
    if not redis_db.get(username):
        print('ERROR: User {0} does not exist'.format(username))
        return False

    new_id = redis_db.incr('post_id')
    post_id = 'post:{0}'.format(new_id)    
    timestamp = date_to_datetime(date_str)

    post_data = {
        'username': username,
        'message': message,
        'timestamp': timestamp
    }
    
    user_id = redis_db.mget(username)[0]

    # 1. Save Post Content
    post_sort_id = 'post:{0}{1}'.format(timestamp, new_id)
    redis_db.hmset(post_sort_id, post_data)

    # 2. Add to users's post list
    posts_user_id = 'posts:{0}'.format(user_id)
    redis_db.lpush(posts_user_id, post_sort_id)
    
    following_user_id = 'following:{0}'.format(user_id)

    for key in redis_db.zscan_iter(following_user_id):
        following_timestamp = key[1]
        following_username = key[0]
        user_id = redis_db.mget(following_username)[0]

        # 3. Add to following user's post list
        posts_user_id = 'posts:{0}'.format(user_id)
        redis_db.lpush(posts_user_id, post_sort_id)

In [12]:
def database_setup():
    relations_df = pd.read_csv('relations.csv')
    users_df = pd.read_csv('twitter_sample.csv')

    for username in relations_df.User.unique():
        nuevo_usuario(username)
        
    for index, row in relations_df.iterrows():
        seguir(row['User'], row['Follows'], row['Following_Time'])
    
    for index, row in users_df.iterrows():
        nuevo_post(row['User'], row['Tweet_Content'], row['Post_Time'])

In [13]:
database_setup()

ERROR: User nan does not exist


In [14]:
def obtener_followers(username):
    user_id = redis_db.mget(username)[0]
    followers_user_id = 'followers:{0}'.format(user_id)
    follower_user_id = ''

    for key in redis_db.zscan_iter(followers_user_id):
        print('{0} at {1}'.format(key[0], datetime.datetime.fromtimestamp(key[1]).isoformat()))

In [15]:
obtener_followers('andyglittle')

charleskod at 2019-07-14T10:07:29
seers_helen at 2019-07-18T09:50:48
karin_stowell at 2019-08-31T15:20:48


In [16]:
def obtener_following(username):
    user_id = redis_db.mget(username)[0]
    followers_user_id = 'following:{0}'.format(user_id)
    follower_user_id = ''

    for key in redis_db.zscan_iter(followers_user_id):
        print('{0} at {1}'.format(key[0], datetime.datetime.fromtimestamp(key[1]).isoformat()))

In [17]:
obtener_following('andyglittle')

hanyshita at 2019-07-12T14:20:46
charleskod at 2019-07-31T12:54:57


In [44]:
def obtener_timeline(username, tweets_propios=False):
    user_id = redis_db.mget(username)[0]
    posts_user_id = 'posts:{0}'.format(user_id)
    sort_by = 'post:*'
    sorted_posts = redis_db.sort(
        posts_user_id,
        by=sort_by,
        get=['*->username', '*->message', '*->timestamp'],
        desc=True
    )
    
    for x in range(0, len(sorted_posts), 3):
        print('@{0}: {1} - at {2}'.format(sorted_posts[x], sorted_posts[x+1], datetime_to_date(float(sorted_posts[x+2]))))
        print('-----')

In [45]:
obtener_timeline('andyglittle')

@andyglittle: Another spot of our #morethanmedicine bus in #bristol this week! If you need support with your cancer diagnosis call us on 0303 3000 118. #livingwellwithcancer https://t.co/eZGLz0BkXB - at 30 Aug 2019 09:55:43
-----
@seers_helen: What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩ #morethanmedicine https://t.co/g2YzMDUpVA - at 30 Aug 2019 00:49:54
-----
@seers_helen: Will you be at #FIX19? Want a preview of @AG_EM33 story? Then check back Monday for #ChangeOfHeart where we sat down with Alin to discuss her new life since her heart #transplant #MoreThanMedicine https://t.co/Xl9zjr7kZ1 - at 29 Aug 2019 19:54:36
-----
@seers_helen: Czy wiesz, że do bez leków dla zwierząt potrzeba by 89% więcej zwierząt do wyprodukowania tej samej ilości mleka i wołowiny?
#MoreThanMedicine https://t.co/SEtHxlV4p1 - at 28 Aug 2019 05:18:35
-----
@andyglittle: For nearly 40 years @PennyBrohnUK has been a link between standard medical treatment and social prescribing, that's why we're at the second @